# Random Walks en Multiplex de Aerolíneas

## Directorio

In [1]:
#Establecer directorio
cd("$(homedir())/Documents/UNAM/Tesis_Lic")

## Paquetes

Se importan todos los paquetes que se necesitarán para las funciones y visualizaciones del proyecto.

In [2]:
using StatsBase

#using NamedArrays
#using CSV
#using DataFrames

In [3]:
using DelimitedFiles

In [4]:
using RandomNumbers.Xorshifts

In [5]:
using LightGraphs

In [6]:
using Plots

In [7]:
using GraphPlot

In [ ]:
#using Compose

In [ ]:
#g = graphfamous("karate")
#gplot(g)

In [ ]:
#draw(PDF("karate.png", 16, 16), gplot(g))

In [ ]:
######### Prueba ejemplo 

#Primero obtenemos la caminata aleatoria para un sólo nodo. 
#Leemos el archivo con la caminata 2B_1_random_walk.csv

archivo_rwalk = readdlm("/Users/yuriko/Downloads/codejulia/data/2B/caminatas/2B_1_random_walk.csv", ',', Int64) 
#Al leer el archivo convierte los números a flotantes

In [9]:
#Leo archivo de diccionario final
archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 

3190×570 Array{Any,2}:
     ""  "global_airports"  …  "ZK_num"  "ZL_num"   "ZM_num"
    1    "AER"                 "NA"      "NA"       "NA"    
    2    "ASF"                 "NA"      "NA"       "NA"    
    3    "CEK"                 "NA"      "NA"       "NA"    
    4    "DME"                 "NA"      "NA"      1        
    5    "EGO"              …  "NA"      "NA"       "NA"    
    6    "GYD"                 "NA"      "NA"       "NA"    
    7    "KGD"                 "NA"      "NA"       "NA"    
    8    "KZN"                 "NA"      "NA"       "NA"    
    9    "LED"                 "NA"      "NA"       "NA"    
   10    "MRV"              …  "NA"      "NA"       "NA"    
   11    "NBC"                 "NA"      "NA"       "NA"    
   12    "NJC"                 "NA"      "NA"       "NA"    
    ⋮                       ⋱                               
 3178    "LPS"                 "NA"      "NA"       "NA"    
 3179    "ORX"                 "NA"      "NA"       "NA"    
 

In [84]:
archivo_dicc[:,494][1212]

253

In [85]:
archivo_dicc[:,2][1212]

"TUS"

In [87]:
archivo_dicc[:,3][1212]

1211

## Funciones

### Importar datos

In [25]:
info_aerolineas = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/nueva_lista_aerolineas.csv", ',', String)
lista_aerolineas = info_aerolineas[:, 2]

568-element Array{String,1}:
 "Airlines"
 "US"      
 "AA"      
 "TK"      
 "DL"      
 "MU"      
 "CZ"      
 "UA"      
 "FR"      
 "BA"      
 "AF"      
 "LH"      
 "CA"      
 ⋮         
 "GY"      
 "EW"      
 "D6"      
 "9D"      
 "7E"      
 "6Y"      
 "6I"      
 "5M"      
 "5G"      
 "4K"      
 "4D"      
 "4B"      

In [26]:
i = 2
airline_code = lista_aerolineas[i]

"US"

In [27]:
aerolinea_matriz = string(airline_code, "_matriz_du.csv")

"US_matriz_du.csv"

In [8]:
"""
    import_matrix(arch::AbstractString)

Función que importa el archivo de la matriz de adyacencia de R.
"""
function import_matrix(arch::AbstractString)
    archivo_R = readdlm(arch, ',') 
    archivo_R = archivo_R[2:end, :]
    matriz = archivo_R[:, 2:end]
    nombre = convert(Array{Int64,2}, matriz)
    return nombre
end

import_matrix

In [28]:
multiplex = @time import_matrix("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")

  0.377313 seconds (634.16 k allocations: 29.164 MiB, 29.40% gc time)


280×280 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

### Nombres de datos

In [10]:
"""
    airport_names(arch::AbstractString)

Función que obtiene un arreglo con los nombres de aeropuertos de la matriz
"""
function airport_names(arch::AbstractString)
    archivo_R = readdlm(arch, ',', String)
    columna = archivo_R[1, :]
    nombres = columna[2:end]
    return nombres
end

airport_names

In [29]:
data_names_multiplex = @time airport_names("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")

  0.157440 seconds (655.30 k allocations: 29.235 MiB, 4.94% gc time)


280-element Array{String,1}:
 "ABI"
 "ABQ"
 "ACT"
 "ALB"
 "ALO"
 "AMA"
 "AMM"
 "AMS"
 "ANC"
 "ANU"
 "ART"
 "ASU"
 "ATH"
 ⋮    
 "AGS"
 "CRW"
 "LYH"
 "PWM"
 "AEX"
 "AGU"
 "GCK"
 "GJT"
 "MGM"
 "SLP"
 "SPI"
 "YKF"

In [62]:
data_names_multiplex[65]

"DFW"

### Grado

In [11]:
"""

    n_in_degree(M::Array{Int64,2}, n::Int64)

Función que obtiene el grado de entrada de un cierto nodo n de la red representada por la matriz de adyacencia M.

Parámetros:
M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}
n nodo a evaluar de tipo Int64

Resultado:
Grado de entrada del nodo n representado por un valor numérico de tipo Int64

Implementación

#### Ejemplos
"""
function n_in_degree(M::Array{Int64,2}, n::Int64)
    grado_entrada = sum(M[n, :])
end

n_in_degree

In [12]:
"""
    in_degree(M::Array{Int64,2})

Función que obtiene el grado de entrada de una red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}

Resultado:
Devuelve un arreglo de tipo Array{Int64,2} que contiene los grados de entrada de todos los nodos n de la red.

## Implementación

### Ejemplos
"""
function in_degree(M::Array{Int64,2})
    grados_entrada = sum(M, dims = 2)
end

in_degree

In [30]:
in_degree_multiplex = @time in_degree(multiplex)

  0.197227 seconds (393.93 k allocations: 19.784 MiB, 3.53% gc time)


280×1 Array{Int64,2}:
 1
 2
 1
 2
 1
 1
 3
 1
 1
 3
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [13]:
"""
    n_out_degree(M::Array{Int64,2}, n::Int64)

Función que obtiene el grado de salida de un cierto nodo n de la red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}
* n nodo a evaluar de tipo Int64

Resultado:
Grado de salida del nodo n representado por un valor numérico de tipo Int64

## Implementación

### Ejemplos
"""

function n_out_degree(M::Array{Int64,2}, n::Int64)
    grado_salida = sum(M[:, n])
end

n_out_degree (generic function with 1 method)

In [14]:
"""
    out_degree(M::Array{Int64,2})

Función que obtiene el grado de salida de una red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}

Resultado:
Devuelve un arreglo de tipo Array{Int64,2} que contiene los grados de salida de todos los nodos n de la red.

## Implementación

### Ejemplos
"""
function out_degree(M::Array{Int64,2})
    grados_salida = sum(M, dims = 1)
end

out_degree

In [31]:
out_degree_multiplex = @time out_degree(multiplex)

  0.014455 seconds (35.74 k allocations: 1.717 MiB)


1×280 Array{Int64,2}:
 1  1  1  2  1  1  3  2  1  3  1  1  1  …  1  1  1  2  1  1  1  1  1  1  1  1

### Distribución de grado

In [15]:
"""
Función de distribución de grado de entrada
"""
function distrib_in_degree(M::Array{Int64,2})
    x = in_degree(M)
    degree_count = counts(x)
    return degree_count
end

distrib_in_degree

In [32]:
distrib_in_degree_multiplex = @time distrib_in_degree(multiplex)

  0.051393 seconds (90.12 k allocations: 4.335 MiB, 13.31% gc time)


156-element Array{Int64,1}:
  12
 118
  39
  31
  17
  17
  10
  10
   9
   2
   3
   1
   1
   ⋮
   0
   0
   0
   0
   0
   0
   0
   0
   0
   0
   0
   1

In [16]:
"""
Función de distribución de grado de salida
"""
function distrib_out_degree(M::Array{Int64,2})
    x = out_degree(M)
    degree_count = counts(x)
    return degree_count
end

distrib_out_degree

In [33]:
distrib_out_degree_multiplex = @time distrib_out_degree(multiplex)

  0.006276 seconds (3.37 k allocations: 154.799 KiB)


146-element Array{Int64,1}:
   5
 122
  41
  30
  20
  16
  14
   7
   8
   1
   4
   1
   1
   ⋮
   0
   0
   0
   0
   0
   0
   0
   0
   0
   0
   0
   1

In [17]:
"""
    plot_distrib(M::Array{Int64,2}, tipo::String, titulo::String, color::String)

Función con parámetros:
M = matriz
tipo = "in" o "out"
titulo = acepta el título del gráfico
color = color de las barras

que regresa un histograma de la distribución de grado, entrada o salida.
"""
function plot_distrib(M::Array{Int64,2}, tipo::String, titulo::String, color::String)
    
    pyplot()
    
    if tipo == "in"
        distribucion = distrib_in_degree(M::Array)
    elseif tipo == "out"
        distribucion = distrib_out_degree(M::Array)
    end
    distribucion
    
    graph = bar(distribucion, fillcolor = color, markersize = 3, linealpha = 0.5, 
    title = titulo, 
    xlabel = "Grado", 
    ylabel = "Frecuencia",
    xlims = (-10,300), #obtener límites
    ylims = (-10,1000)) #obtener límites
end

plot_distrib

### Vecinos

In [18]:
"""
    neighbours(M::Array{Int64,2})

Función que obtiene un conjunto de arreglos con los vecinos de cada nodo.
"""
function neighbours(M::Array{Int64,2})
    
    nodos = size(M, 1) #aeropuertos
    vecindades = []
    
    for i in 1:nodos
        vecinos = findall(x -> x != 0, M[i, :])
        push!(vecindades, vecinos)
    end
    vecindades
end

neighbours

In [34]:
neighbors_multiplex = @time neighbours(multiplex)

  0.106300 seconds (207.80 k allocations: 10.605 MiB, 5.79% gc time)


280-element Array{Any,1}:
 [65]           
 [65, 180]      
 [65]           
 [47, 63]       
 [180]          
 [65]           
 [117, 180, 263]
 [189]          
 [190]          
 [47, 117, 156] 
 [180]          
 [156]          
 [189]          
 ⋮              
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        
 Int64[]        

### Random walks

In [19]:
"""

    r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)

La función r_walks tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
f, nodo inicial de tipo Int64
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz, el cual tiene igual probabilidad de avanzar 
hacia cualquiera de sus vecinos.
Regresa un arreglo que contiene todas las trayectorias que tomó el caminante aleatorio.
"""
function r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)
    
    paths = zeros(Int64, (iter, steps + 1))
    s = f
    seed = 1
    vecindades = neighbours(M)

    for i in 1:iter
        paths[i, 1] = f
        
        for p in 1:steps 
            vecindad = vecindades[s]
            
            if vecindad == []
                break
            else
                a = sample(Xoroshiro128Plus(seed), vecindad) 
                paths[i, p + 1] = a
                s = a
                seed += 1
            end
            
        end
        s = f
        
    end
    
    return paths 
end

r_walks

In [20]:
"""

    random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64)

La función random_walk tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz con igual probabilidad de avanzar 
hacia cualquiera de sus vecinos. Esta función recorre todos los nodos de la red como
nodos iniciales y sobre ellos itera para obtener el número de caminos que se le pidió.
Regresa un arreglo que contiene arreglos asociados a todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""
function random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64) 
    
    random_walks = []
    nodos = size(M, 1)
    
    for f in 1:nodos #aeropuertos
        paths = r_walks(M, f, steps, iter)
        push!(random_walks, paths)
    end
    
    random_walks
end

random_walk

In [90]:
random_walk_multiplex = @time random_walk(multiplex, 5, 2)

  0.141005 seconds (579.68 k allocations: 205.648 MiB, 25.82% gc time)


280-element Array{Any,1}:
 [1 65 … 219 65; 1 65 … 64 124]     
 [2 180 … 124 48; 2 65 … 64 124]    
 [3 65 … 219 65; 3 65 … 64 124]     
 [4 63 … 193 65; 4 47 … 184 65]     
 [5 180 … 124 48; 5 180 … 184 65]   
 [6 65 … 219 65; 6 65 … 64 124]     
 [7 263 … 189 57; 7 117 … 184 65]   
 [8 189 … 190 216; 8 189 … 184 65]  
 [9 190 … 47 108; 9 190 … 144 65]   
 [10 156 … 193 65; 10 47 … 184 65]  
 [11 180 … 124 48; 11 180 … 184 65] 
 [12 156 … 193 65; 12 156 … 27 81]  
 [13 189 … 190 216; 13 189 … 184 65]
 ⋮                                  
 [269 0 … 0 0; 269 0 … 0 0]         
 [270 0 … 0 0; 270 0 … 0 0]         
 [271 0 … 0 0; 271 0 … 0 0]         
 [272 0 … 0 0; 272 0 … 0 0]         
 [273 0 … 0 0; 273 0 … 0 0]         
 [274 0 … 0 0; 274 0 … 0 0]         
 [275 0 … 0 0; 275 0 … 0 0]         
 [276 0 … 0 0; 276 0 … 0 0]         
 [277 0 … 0 0; 277 0 … 0 0]         
 [278 0 … 0 0; 278 0 … 0 0]         
 [279 0 … 0 0; 279 0 … 0 0]         
 [280 0 … 0 0; 280 0 … 0 0]         

### Nodos/aeropuertos visitados

In [120]:
"""
    visited_vertices(W::Array{Any,1})

La función caminos tiene como parámetro W que es un arreglo de arreglos.
Regresa los nodos diferentes a los que llega el caminante.
"""
function visited_vertices(W::Array{Any,1})
    
    cobert = Array[]
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos, no necesario
    #visit = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    visit = Array[]
    
    for i in 1:v        
        for j in 1:s
            u = unique(W[i][j, :])
            push!(visit, u)
        end
        
        push!(cobert, visit)
        visit = []
    end
    
    return cobert
end

visited_vertices

In [102]:
visited_vertices(random_walk_multiplex)[1]

2-element Array{Array,1}:
 [1, 65, 253, 219]         
 [1, 65, 176, 190, 64, 124]

### Random walks con nodos globales

In [224]:
"""
    walk_global(W::Array{Any,1}, airline_code::String, v::Int64)

Función que cambia las caminatas del número de nodo asociado a la aerolínea a número de nodo GLOBAL de aeropuertos.
Parámetros:
W = caminatas aleatorias (arreglo de matrices)
airline_code = código de la aerolínea
v = nodo inicial
"""
function walk_global(W::Array{Any,1}, airline_code::String, v::Int64)
    
    #Ahora se usa la función para obtener los nodos diferentes
    nodos_diferentes = visited_vertices(W)
    
    #vv2b[1][1,:] #Nodos únicos visitados en la iteración 1 con nodo inicial 1 #Leo los visited vertices del archivo
    m = size(W[1], 1) #renglones/iteraciones
    #@show m
    n = size(W[1], 2) #columnas/pasos+1/nodos
    #@show n
    l = length(W) #número de nodos iniciales
    #@show l
    
    #Se lee el diccionario
    archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 
    #@show archivo_dicc
    aerol_col = string(airline_code, "_num") #Nombre de la columna
    coord_aerol = findall(x -> x == aerol_col, archivo_dicc) #Busco las coordenadas de la columna
    col_aerol = coord_aerol[1][2] #Segunda coordenada de la posición (columna)
    columna_aerol = archivo_dicc[:, col_aerol] #Obtengo la columna con el nombre del aeropuerto al que corresponde
    #@show columna_aerol
    walk_empty = zeros(Int64, m, n)#Arreglo vacío para nuevos números
        #@show walk_empty
    
    for k = v #1:l #nodo inicial v
        #@show k
        arreglo_visitas = nodos_diferentes[k]
        #@show arreglo_visitas
        caminatas_nodo = W[k] #caminata y sus iteraciones con el mismo nodo inicial
        #@show caminatas_nodo
        
        
        for i in 1:m #voy por las iteraciones
            #@show i
            nodos_dif = arreglo_visitas[i, :] #array/caminata única de los nodos que buscará
            #@show nodos_dif
            
            for j in 1:length(nodos_dif[]) #vamos por los nodos diferentes por los que pasé
                #@show j
                
                nodo_original = nodos_dif[][j] #index original de la aerolínea
                #@show nodo_original
                
                #Busco la posición original del aeropuerto que se asignará
               # if 
                posicion_original = findall(x -> x == nodo_original, archivo_dicc[:, col_aerol]) 
                #Este arreglo tendrá la coordenada de la posición del nodo inicial, sin embargo, si llegué a un endpoint, 
                #buscará la coordenada de la posición del nodo 0 y te regresará un un arreglo vacío.
                #@show posicion_original
                #Se busca el nombre del aeropuerto
                
                if isempty(posicion_original) == false
                    
                    numero_asignado = archivo_dicc[posicion_original, 3]
                    #@show numero_asignado
                    #Busco el primer nodo diferente en la iteración i
                    
                    coords_ys = findall(x -> x == nodo_original, caminatas_nodo[i, :]) #Encuentra uno de los nodos dif en la iteración
                    #@show coords_ys
                    
                    for q in 1:length(coords_ys) #Se itera sobre las posiciones en las caminatas
                        #@show length(coords_ys)
                        #@show q
                        
                        pos_actual = coords_ys[q]
                        #nodo_original = findall(x -> x == pos_actual, columna_aerol) #buscar el nodo en la aerolínea
                        #aerop_nombre = nodo_original[1]+1
                        #aeropuerto_asignado = archivo_dicc[nodos_dif[i],2]#nodo_original, 2] #buscando qué nombre se colocará
                        
                        walk_empty[i, pos_actual] = numero_asignado[] #asigno valor de diccionario al arreglo vacío i, pos_actual
                        #@show walk_empty
                    end
                
                elseif break
                end
                
            end
            #@show walk_empty

        end
        #@show walk_empty
        
        
    end
    
    return walk_empty
   
end

walk_global

In [205]:
caso = random_walk_multiplex[269]

2×6 Array{Int64,2}:
 269  0  0  0  0  0
 269  0  0  0  0  0

In [208]:
archivo_dicc[:, 494]

3190-element Array{Any,1}:
    "US_num"
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
   ⋮        
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
    "NA"    
 273        
 275        
 277        

In [217]:
hu = findall(x -> x == 0, archivo_dicc[:, 494])

0-element Array{Int64,1}

In [226]:
for v = 269
    walk_global_multiplex = @time walk_global(random_walk_multiplex, airline_code, v)
    return walk_global_multiplex
end

  1.732556 seconds (12.59 M allocations: 460.010 MiB, 63.36% gc time)


2×6 Array{Int64,2}:
 1100  0  0  0  0  0
 1100  0  0  0  0  0

In [221]:
function walks_global(W::Array{Any,1}, airline_code::String)
    
    t = length(W)
    #@show t
    
    arreglo = Array[]
    
    for v in 267:t
        @show v
        #archivo_caminata_global = string(airline_code, "_", v,"_walk_global.csv")
        #@show archivo_caminata_global
        
        walk_global_multiplex = walk_global(W, airline_code, v)
        #return walk_global_multiplex
        push!(arreglo, walk_global_multiplex)
        #writedlm("/Users/Yuriko/Desktop/carpetita/$archivo_caminata_global", walk_global_multiplex, ',')
    end
    return arreglo
end

walks_global (generic function with 1 method)

In [222]:
walks_global(random_walk_multiplex, airline_code)

v = 267
j = 1
nodo_original = 267
posicion_original = [417]
numero_asignado = Any[416]
j = 2
nodo_original = 190
posicion_original = [1186]
numero_asignado = Any[1185]
j = 3
nodo_original = 192
posicion_original = [1187]
numero_asignado = Any[1186]
j = 4
nodo_original = 124
posicion_original = [339]
numero_asignado = Any[338]
j = 5
nodo_original = 47
posicion_original = [716]
numero_asignado = Any[715]
j = 6
nodo_original = 108
posicion_original = [1151]
numero_asignado = Any[1150]
j = 1
nodo_original = 267
posicion_original = [417]
numero_asignado = Any[416]
j = 2
nodo_original = 190
posicion_original = [1186]
numero_asignado = Any[1185]
j = 3
nodo_original = 145
posicion_original = [1168]
numero_asignado = Any[1167]
j = 4
nodo_original = 65
posicion_original = [1131]
numero_asignado = Any[1130]
j = 5
nodo_original = 144
posicion_original = [1167]
numero_asignado = Any[1166]
v = 268
j = 1
nodo_original = 268
posicion_original = [59]
numero_asignado = Any[58]
j = 2
nodo_original = 225


BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index []

In [165]:
for v = 1:length(random_walk_multiplex)
    @show v
    archivo_caminata_global = string(airline_code, "_", v,"_walk_global.csv")
    @show archivo_caminata_global
	walk_global_multiplex = @time walk_global(random_walk_multiplex, airline_code, v)
    return walk_global_multiplex
    writedlm("/Users/Yuriko/Desktop/carpetita/$archivo_caminata_global", walk_global_multiplex, ',')
end

v = 1
archivo_caminata_global = "US_1_walk_global.csv"
  1.702261 seconds (12.64 M allocations: 461.234 MiB, 64.03% gc time)


2×6 Array{Int64,2}:
 2875  1130  1211  1130  1197  1130
 2875  1130  1176  1185  1129   338

### Random walks con nombres

In [227]:
"""
    walk_names(W::Array{Any,1}, airline_code::String, v::Int64)

Función que cambia las caminatas del número de nodo asociado a la aerolínea a nombres de aeropuertos.
Parámetros:
W = caminatas aleatorias (arreglo de matrices)
airline_code = código de la aerolínea
v = nodo inicial
"""
function walk_names(W::Array{Any,1}, airline_code::String, v::Int64) 
    
    #Primero se lee el archivo con las caminatas
    #path = string("/Users/yuriko/Downloads/codejulia/data/",airline_code,"/",airline_code,"_",v,"_random_walk.csv")
    #W = readdlm("/Users/yuriko/Downloads/codejulia/data/2B/caminatas/2B_1_random_walk.csv", ',', Int64) 
    
    #Ahora se usa la función para obtener los nodos diferentes
    nodos_diferentes = visited_vertices(W)
    #vv2b[1][1,:] #Nodos únicos visitados en la iteración 1 con nodo inicial 1 #Leo los visited vertices del archivo
    m = size(W[1], 1) #renglones
    n = size(W[1], 2) #columnas
    l = length(W) #número de nodos iniciales
    
    #Se lee el diccionario
    archivo_dicc = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/diccionarios/dictionary_final.csv", ',') 
    
    aerol_col = string(airline_code, "_num") #Nombre de la columna
    coord_aerol = findall(x -> x == aerol_col, archivo_dicc) #Busco las coordenadas de la columna
    col_aerol = coord_aerol[1][2] #Segunda coordenada de la posición (columna)
    columna_aerol = archivo_dicc[:, col_aerol] #Obtengo la columna con el nombre del aeropuerto al que corresponde
    walk_nothing = Array{Union{Nothing, String}}(nothing, m, n) #archivo vacío de una matriz para un nodo inicial
    
    for k = v #1:l #nodo inicial v
        arreglo_visitas = nodos_diferentes[k]
        caminatas_nodo = W[k]
        
        for i in 1:m #voy por las iteraciones
            nodos_dif = arreglo_visitas[i, :] #array/caminata única de los nodos que buscará
            
            for j in 1:length(nodos_dif[])
                nodo_original = nodos_dif[][j]
                #Busco la posición original del aeropuerto que se asignará
                posicion_original = findall(x -> x == nodo_original, archivo_dicc[:, col_aerol])
                #Se busca el nombre del aeropuerto
                
                if isempty(posicion_original) == false
                        
                    nombre_asignado = archivo_dicc[posicion_original, 2]
                    #Busco el primer nodo diferente en la iteración i
                    coords_ys = findall(x -> x == nodo_original, caminatas_nodo[i, :]) #Encuentra uno de los nodos dif en la iteración
                    
                    for q in 1:length(coords_ys) #Se itera sobre las posiciones en las caminatas
                        pos_actual = coords_ys[q]
                        #nodo_original = findall(x -> x == pos_actual, columna_aerol) #buscar el nodo en la aerolínea
                        #aerop_nombre = nodo_original[1]+1
                        #aeropuerto_asignado = archivo_dicc[nodos_dif[i],2]#nodo_original, 2] #buscando qué nombre se colocará
                        walk_nothing[i, pos_actual] = nombre_asignado[] #asigno valor de diccionario al arreglo vacío i, pos_actual
                    end
                    
                elseif break
                end
            end

        end
        
    end
    return walk_nothing
end

walk_names

In [230]:
for k = 269#:length(random_walk_multiplex)
		walk_names_multiplex = @time walk_names(random_walk_multiplex, airline_code, k)
    return walk_names_multiplex
end

  1.721403 seconds (12.59 M allocations: 460.010 MiB, 63.05% gc time)


2×6 Array{Union{Nothing, String},2}:
 "AGS"  nothing  nothing  nothing  nothing  nothing
 "AGS"  nothing  nothing  nothing  nothing  nothing

### Longitud de caminatas

In [ ]:
"""
    walk_length(W::Array{Any,1})

La función walk_length tiene como parámetros:
RW, arreglo de arreglos de tipo Array{Any,1} que corresponde al arreglo de caminatas aleatorias.
La función regresa un arreglo de arreglos que contiene las longitudes de todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""
function walk_length(W::Array{Any,1})
    
    longitudes = Array[]
    
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #Es el número de iteraciones
    longitud = zeros(Int64, s)
    
    for i in 1:v
        
        for j in 1:s
               
            longitud[j] = findlast(x -> x != 0, W[i][j, :])
            
        end
         push!(longitudes, longitud)
        
        longitud = zeros(Int64, s)
    end
    return longitudes
        
end

In [ ]:
walk_length(caminata_M)[1]

In [ ]:
"""
    walk_av_length(W::Array{Any,1})

La función walk_av_length tiene como parámetro W, que es un arreglo de arreglos (que se puede obtener de la 
función walk_length). 
Regresa un arreglo con entradas correspondientes al promedio de las longitudes de 
las caminatas aleatorias para cada nodo de inicio antes de caer en un nodo con grado de salida 0.
"""
function walk_av_length(W::Array{Any,1}) #Llamar a función
    
    proms = Array{Float64}(undef, 0)
    wlength = walk_length(W)
    
    for i in 1:length(wlength)
        av = mean(wlength[i])
        push!(proms, av)
    end
    proms
end

In [ ]:
walk_av_length(caminata_M)

In [ ]:
"""
    plot_av_length_vertex(W::Array{Float64, 1}, airline_code::String)

Función que grafica (scatter plot) del promedio por nodo inicial de caminatas
"""
function plot_av_length_vertex(W::Array{Any,1}, airline_code::String)
    
    pyplot()
    
    info = walk_av_length(W)
    #@show info
    
    #dc = nombres_multiplex
    @show data_names_multiplex
    scatter(info, 
        color = "#0f87bf",
        alpha = 0.8, 
        title = airline_code, 
        xticks = (1:1:length(W), data_names_multiplex),
        xlabel = "Aeropuerto inicial", 
        ylabel = "Longitud",
        legendtitle = airline_code,
        label = "Promedio")
end

In [ ]:
plot_av_length_vertex(caminata_M, "2B")

In [ ]:
"""
    walk_layer_length(W::Array{Any,1})

Función que saca el promedio de longitud de caminata en la capa
"""
function walk_layer_length(W::Array{Any,1})
    
    x = walk_av_length(W)
    av = mean(x)
    
    return av
end

In [ ]:
walk_layer_length(caminata_M)

### Frecuencia de visitas

In [ ]:
"""
    freq_vertices_visits(W::Array{Any,1})

Función que hace un diccionario con frecuencias de visitas
"""
function freq_vertices_visits(W::Array{Any,1})
    
    dicc = [] #arreglo vacío
    
    nodos = length(W) #tamaño de caminatas
    iter = size(W[1], 1) #núm de iteraciones
    diccionario = 0
    
    #for i = n
    for i in 1:nodos #corro por todos los nodos
        for j in 1:iter #corro por las iteraciones
            diccionario = countmap(W[i][j, :]) #hago diccionario de freqs de cada caminata
            push!(dicc, diccionario) #guardo el diccionario en un arreglo vacío
        end
        
        diccionario = 0
    end
    return dicc
end

In [ ]:
"""
    distrib_vertices_visits(W::Array{Any,1})

Función que crea un arreglo de arreglos con la distribución de frecuencia de visitas
"""
function distrib_vertices_visits(W::Array{Any,1})
    
    x = freq_vertices_visits(W) #diccionario de frecuencias
    l = length(W)
    f = 0
    frec_total = Array[] #zeros(Array, length(x)) #arreglo de ceros de tamaño núm dicc
    frec_nodo = zeros(Int64, l+1) #arreglo de nodos de tamaño núm nodos para get frec

    for i in 1:length(x) #corro por los nodos

        for j in 1:l+1 #corro por los nodos

            frec_nodo[j] = get(x[i], f, 0) #entrada j de freq nodo la cambio por la entrada j del dicc i
            f += 1
        end
        push!(frec_total, frec_nodo)
        
        f = 0
        frec_nodo = zeros(Int64, l+1)
        
    end
    
    return frec_total
end

In [ ]:
############# FUNCIÓN INÚTIL #############
"""
    plot_visit_freq(W::Array{Any,1})

Función que hace un heatmap de nodos en los que cae, cada renglón corresponde a una caminata
"""
function plot_visit_freq(W::Array{Any,1}) 
    
    pyplot()
    x = distrib_vertices_visits(W)
    h = transpose(hcat(x...))

    xs = [string("n", i) for i = 0:1:length(W)]
    ys = [string("rw", i) for i = 1:length(x)]
    z = h
    heatmap(xs, ys, z, aspect_ratio = 1)
end

In [ ]:
"""
    freq_vertex_visits(W::Array{Any,1}, n::Int64)

Función que hace un diccionario con frecuencias para un nodo inicial
"""
function freq_vertex_visits(W::Array{Any,1}, n::Int64) #puede ser más eficiente llamando la función anterior sin nodo
    
    dicc = [] #arreglo vacío
    
    nodos = length(W) #tamaño de caminatas
    iter = size(W[1], 1) #núm de iteraciones
    diccionario = 0
    
    for j in 1:iter #corro por las iteraciones
          
        diccionario = countmap(W[n][j, :]) #hago diccionario de freqs de cada caminata
          
        push!(dicc, diccionario) #guardo el diccionario en un arreglo vacío
          
    end
    
    return dicc
end

In [ ]:
"""
    distrib_vertex_visits(W::Array{Any,1}, n::Int64)

Función que crea un arreglo de arreglos con la distribución de frecuencias de visitas de un nodo inicial
"""
function distrib_vertex_visits(W::Array{Any,1}, n::Int64)
    
    x = freq_vertex_visits(W, n) #diccionario de frecuencias
    nodos = length(W)
    f = 0
    frec_total = [] #zeros(Array, length(x)) #arreglo de ceros de tamaño núm dicc
    frec_nodo = zeros(Int64, nodos+1) #arreglo de nodos de tamaño núm nodos para get frec

    for i in 1:length(x) #corro por los nodos

        for j in 1:nodos+1 #corro por los nodos

            frec_nodo[j] = get(x[i], f, 0) #entrada j de freq nodo la cambio por la entrada j del dicc i
            
            f += 1
        end
        push!(frec_total, frec_nodo)
        
        f = 0
        frec_nodo = zeros(Int64, nodos+1)
        
    end
    
    return frec_total
end

In [ ]:
"""
    plot_visit_freq_vertex(W::Array{Any,1}, n::Int64)

Función que hace un heatmap de los nodos en los que cae, cada renglón corresponde a una caminata 
con cierto nodo inicial n.
"""
function plot_visit_freq_vertex(W::Array{Any,1}, n::Int64) #Con esta función se pierde la información de los nodos que caen en 0
    
    pyplot()
    
    x = distrib_vertex_visits(W, n)
    mat_completa = transpose(hcat(x...))    #Se covierte en matriz el arreglo
    mat = mat_completa[:, (2:end)]
    pasos = size(W[1], 2)-1
    
    xs = [nombres_multiplex]
    ys = [string("it", i) for i = 1:length(x)]
    z = mat
    heatmap(xs, ys, z, 
        title = "Frecuencia de visitas para v_{0} = $n",
        aspect_ratio = 1, 
        clims = (0, pasos), 
        xticks = :all,  
        xtickfont = font(8, "Courier"))
end

In [ ]:
"""
    distrib_av_visits(W::Array{Any, 1})

Función que hace una distribución de visitas por nodo inicial y muestra todos los nodos iniciales
"""
function distrib_av_visits(W::Array{Any, 1}) 
    
    nodos = length(W) #número de nodos
    iteracion = size(W[1],1)
    dicc = zeros(Float64, nodos, nodos+1) #matriz de ceros (nodos x nodos+1)
    
    for i in 1:nodos #(1 a 17)
        prom_conteos = counts(W[i], 0:nodos)/iteracion #hago diccionario con prom de freqs de cada nodo inicial
        #@show prom_conteos
        dicc[i, :] = prom_conteos 
    end

    dicc
end

In [ ]:
"""
    plot_freq_av_visits(W::Array{Any, 1})

Función que crea un diccionario con frecuencias por nodo inicial y muestra todos
"""
function plot_freq_av_visits(W::Array{Any, 1}, airline_code::String)
    
    pyplot()
    
    mat_completa = distrib_av_visits(W)
    mat = mat_completa[:, (2:end)] #desde la segunda columna para no tomar en cuenta los end-points
    
    xs = [nombres_multiplex]
    ys = [nombres_multiplex] #[string("v_0 ", i) for i = 1:length(W)]
    z = mat
    
    heatmap(xs, ys, z, 
        title = string("Frecuencia de visitas para ", airline_code),
        aspect_ratio = 1, 
        clims = (0, maximum(mat)), #MÁXIMO DEL MULTIPLEX GLOBAL/AEROLÍNEA MÁS GRANDE ¿?
        xticks = :all,
        yticks = :all,
        xtickfont = font(7, "Courier"),
        ytickfont = font(7, "Courier"),
        xlabel = "Visitas promedio",
        ylabel = "Aeropuerto inicial",
        xrotation = 50)
end

In [ ]:
plot_freq_av_visits(caminata_M, "2B")

### Cobertura

In [ ]:
"""
    coverage(W::Array{Any,1})

La función cobertura tiene como parámetro W que es un arreglo de arreglos.
Regresa el número de nodos diferentes a los que llega el caminante.
"""
function coverage(W::Array{Any,1}) 
    
    cobert = []
    
    v = length(W) #tamaño de la matriz, número de nodos
    x = visited_vertices(W)
    s = size(x[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v        
        for j in 1:s
            l = length(x[i][j])
            longz[j] = l
        end
        
        push!(cobert, longz)
        longz = zeros(Int64, s)
    end
    
    return cobert
end

In [ ]:
"""
    vertex_coverage(W::Array{Any,1})

Da el promedio de cobertura la caminata por nodo inicial. 
"""
function vertex_coverage(W::Array{Any,1}) #Esta es la importante
    
    longitud = Array{Float64}(undef, 0)
    x = coverage(W)
    
    for i in 1:length(x)
        l = mean(x[i])
        push!(longitud, l)
    end
    
    longitud
end

In [ ]:
vertex_coverage(caminata_M)

In [ ]:
"""
    plot_vertex_coverage(W::Array{Any, 1}, airline_code::String)

Función que grafica (scatter plot) del promedio por nodo inicial de caminatas
"""
function plot_vertex_coverage(W::Array{Any, 1}, airline_code::String)
    
    info = vertex_coverage(W)
    
    scatter(info, 
        color = "#0f87bf",
        alpha = 0.4, 
        title = string("Cobertura por nodo inicial ", airline_code), 
        xlabel = "Aeropuerto inicial", 
        ylabel = "Longitud", 
        xticks = (1:1:length(W), nombres_multiplex),
        legendtitle = airline_code,
        label = "Promedio",
        xrotation = 50)
    
end

In [ ]:
plot_vertex_coverage(caminata_M, "2B")

In [ ]:
"""
    layer_coverage(W::Array{Any, 1})

Da el promedio de la longitud de las caminatas para todos los nodos, como un número.
"""
function layer_coverage(W::Array{Any, 1})
    x = vertex_coverage(W)
    promedio = mean(x)
    return promedio
end

### Máximos y mínimos de longitud

In [ ]:
"""
    max_length(W::Array{Any, 1})

Da la longitud máxima de cada iteración de cada nodo inicial.
"""
function max_length(W::Array{Any, 1})
    max = Array{Int64}(undef, 0)
    x = coverage(W)
    for i in 1:length(x)
        m = maximum(x[i])
        push!(max, m)
    end
    max
end

In [ ]:
"""
    Max_length(W::Array{Any, 1})

Da la longitud máxima total.
"""
function Max_length(W::Array{Any, 1})
    x = max_length(W)
    m = maximum(x)
    return m
end

In [ ]:
"""
    av_max_length(W::Array{Any, 1})

Regresa el promedio de la longitud máxima de cada nodo inicial.
"""
function av_max_length(W::Array{Any, 1})
    x = max_length(W)
    prom = mean(x)
    return prom
end

In [ ]:
"""
    min_length(W::Array{Any, 1})

Da la longitud mínima de cada iteración de cada nodo inicial.
"""
function min_length(W::Array{Any, 1})
    min = Array{Int64}(undef, 0)
    x = coverage(W)
    for i in 1:length(x)
        m = minimum(x[i])
        push!(min, m)
    end
    min
end

In [ ]:
"""
    Min_length(W::Array{Any, 1})

Da la longitud mínima total.
"""
function Min_length(W::Array{Any, 1})
    x = min_length(W)
    m = minimum(x)
    return m
end

In [ ]:
"""
    av_min_length(W::Array{Any, 1})

Regresa el promedio de la longitud mínima de cada nodo inicial.
"""
function av_min_length(W::Array{Any, 1})
    x = min_length(W)
    prom = mean(x)
    return prom
end

### End-points

In [ ]:
"""
    endpoints(W::Array{Any, 1})

Regresa un arreglo de arreglos que contiene los nodos en los que cae cada caminata y ya no puede salir.
"""
function endpoints(W::Array{Any, 1})
    
    x = visited_vertices(W)
    
    atractores = Array[] #Arreglo vacío que contenga a los atractores
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos +1
    atract = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v        
        for j in 1:s
            if x[i][j, :][end][end] == 0
                at = x[i][j, :][end][end - 1]
                atract[j] = at
            else
                atract[j] = 0
            end
        end
        push!(atractores, atract)
        atract = zeros(Int64, s)
    end
    
    return atractores
end

In [ ]:
hholi =endpoints(caminata_M)

In [ ]:
"""
   which_endpoints(M::Array{Int64,2})

Función que da el índice de los end-points.
"""
function which_endpoints(M::Array{Int64,2})
    deg = out_degree(M)
    ci = findall(iszero, deg)
    arr = Array{Int64}(undef, 0)
    for i in 1:length(ci)
        index = ci[1][2]
        push!(arr, index)
    end
    return arr
end

In [ ]:
"""
    freq_all_endpoints(W::Array{Any, 1})

Función con diccionarios de frecuencias de end-points
"""
function freq_all_endpoints(W::Array{Any, 1})
    
    info = endpoints(W)
    resu = []
    c = 0
    
    for i in 1:length(info)
    
        c = countmap(info[i])
        push!(resu, c)
        c = 0
    end
    return resu
end

In [ ]:
freq_all_endpoints(caminata_M)

In [ ]:
function distrib_all_endpoints(M::Array{Int64,2}, W::Array{Any, 1})
    
    info = freq_all_endpoints(W)
    endpts = which_endpoints(M)
    ######################################
    
    l = length(W) #num nodos iniciales
    e = length(endpts) 
    #@show l
    f = 0  #contador
    frec_total = Array[] #zeros(Array, length(x)) #arreglo de ceros de tamaño núm dicc
    frec_nodo = zeros(Int64, l+1) #arreglo de nodos de tamaño núm nodos para get frec

    for i in 1:length(info) #corro por los nodos

        for j in 1:l+1 #corro por los nodos

            frec_nodo[j] = get(info[i], f, 0) #entrada j de freq nodo la cambio por la entrada j del dicc i
            f += 1
        end
        push!(frec_total, frec_nodo)
        
        f = 0
        frec_nodo = zeros(Int64, l+1)
        
    end
    
    return frec_total
    
end

In [ ]:
distrib_all_endpoints(caminata_M)

In [ ]:
out_degree_multiplex[15] = 0
out_degree_multiplex

In [ ]:
which_endpoints(out_degree_multiplex)

In [ ]:
#función que cambie los endpoints a nombres de aeropuertos

In [ ]:
""" NOOOO
    plot_endpoints(W::Array{Any, 1}, airline_code::String)

Función que grafica la información de los endpoints por nodo inicial
"""
function plot_endpoints(W::Array{Any, 1}, airline_code::String) #No necesario y mal
    
    pyplot()
    
    info = endpoints(W)
    
    mat_completa = distrib_av_visits(W)
    mat = mat_completa[:, (2:end)] #desde la segunda columna para no tomar en cuenta los end-points
    
    xs = [nombres_multiplex]
    ys = [nombres_multiplex] #[string("v_0 ", i) for i = 1:length(W)]
    z = mat
    
    heatmap(xs, ys, z, 
        title = string("Frecuencia de visitas para ", airline_code),
        aspect_ratio = 1, 
        clims = (0, maximum(mat)), #MÁXIMO DEL MULTIPLEX GLOBAL/AEROLÍNEA MÁS GRANDE ¿?
        xticks = :all,
        yticks = :all,
        xtickfont = font(7, "Courier"),
        ytickfont = font(7, "Courier"),
        xlabel = "Visitas promedio",
        ylabel = "Aeropuerto inicial",
        xrotation = 50)
end

In [ ]:
plot_endpoints(caminata_M, "2B")

In [ ]:
"""
    av_vertex_endpoints(W::Array{Any, 1})

Función que calcula el promedio de endpoints por nodo
"""
function av_vertex_endpoints(W::Array{Any, 1}) 
    
    x = endpoints(W)
    atractores_prom = Array{Float64}(undef, 0) #Arreglo vacío que contenga a los atractores
    v = length(W) #tamaño de la matriz, número de nodos

    for i in 1:v        
        av = mean(x[i])
        push!(atractores_prom, av)
    end
    
    atractores_prom
end

In [ ]:
av_vertex_endpoints(caminata_M)

### Distribución de frecuencia de end-points

In [ ]:
"""
    freq_endpoints(W::Array{Any, 1})

Función que da un diccionario con frecuencias
"""
function freq_endpoints(W::Array{Any, 1})
    freqs = []
    x = endpoints(W)
    v = length(W)
    
    for i in 1:v
        frec = countmap(x[i])
        push!(freqs, frec)
    end
    freqs
end

In [ ]:
freq_endpoints(caminata_M)

In [ ]:
"""
    distrib_endpoints(W::Array{Any, 1})

Función que saca la distribución de los endpoints para toda la capa
"""
function distrib_endpoints(W::Array{Any, 1})
    
    x = freq_endpoints(W)
    frec_total = zeros(Int64, length(W))
    frec_nodo = zeros(Int64, length(W))

    for i in 1:length(W)
        for j in 1:length(W)
            frec_nodo[j] = get(x[i], j, 0)
        end
        frec_total += frec_nodo
    end
    
    return frec_total
end

In [ ]:
distrib_endpoints(caminata_M)

### Gráficas de end-points

In [ ]:
"""
    plot_vertex_endpoints(W::Array{Any, 1}, airline_code::String)

Función que grafica la información de los endpoints por nodo inicial
"""
function plot_vertex_endpoints(W::Array{Any, 1}, airline_code::String) #MAL
    
    info = distrib_endpoints(W) #endpoints por nodo inicial
    
    heatmap(xs, ys, z, 
        title = string("Frecuencia de visitas para ", airline_code),
        aspect_ratio = 1, 
        clims = (0, maximum(mat)), #MÁXIMO DEL MULTIPLEX GLOBAL/AEROLÍNEA MÁS GRANDE ¿?
        xticks = :all,
        yticks = :all,
        xtickfont = font(7, "Courier"),
        ytickfont = font(7, "Courier"),
        xlabel = "Visitas promedio",
        ylabel = "Aeropuerto inicial",
        xrotation = 50)
    
    scatter(info, 
        color = "#0f87bf",
        alpha = 0.4, 
        title = string("End-points por nodo inicial ", airline_code), 
        xlabel = "Aeropuerto inicial", 
        ylabel = "End-points", 
        xticks = (1:1:length(W), nombres_multiplex),
        legendtitle = airline_code,
        label = "Promedio",
        xrotation = 50)
end

In [ ]:
distrib_endpts_M = @time distrib_endpoints(caminata_M)

In [ ]:
"""
    plot_layer_endpoints(W::Array{Any, 1})

Función que grafica la información de los endpoints en toda una capa(aerolínea)
"""
function plot_layer_endpoints(W::Array{Any, 1})
    
end

### Distribución Betweenness

In [ ]:
"""
    plot_betweenness(W::Array{Any, 1}, airline_code::String)

Función que grafica (scatter plot) de la distribución de betweenness de los nodos.
"""
function plot_betweenness()
    
    
end


function plot_vertex_coverage(W::Array{Any, 1}, airline_code::String)
    
    info = vertex_coverage(W)
    
    scatter(info, 
        color = "#0f87bf",
        alpha = 0.4, 
        title = string("Cobertura por nodo inicial ", airline_code), 
        xlabel = "Aeropuerto inicial", 
        ylabel = "Longitud", 
        xticks = (1:1:length(W), nombres_multiplex),
        legendtitle = airline_code,
        label = "Promedio",
        xrotation = 50)
    
end

### Grafo

In [ ]:
"""
    my_graph(M::Array{Int64,2}, airline_code::String)

Función que hace el grafo de la red aérea dirigida.
"""
function my_graph(M::Array{Int64,2}, airline_code::String)
    
    grafo = DiGraph(M)
    
end

In [ ]:
jh= my_graph(multiplex, "2B")

### Plot Grafo

In [ ]:
"""
    plot_net(M::Array{Int64,2}, airline_code::String)

Función que hace el grafo de la red aérea dirigida.
"""
function plot_net(M::Array{Int64,2}, airline_code::String)
    
    grafo = DiGraph(M)
    red = gplot(grafo)
    
    return red
    
end

In [ ]:
ih= plot_net(multiplex, "2B")

In [ ]:
"""
    my_betweenness_centrality(M::Array{Int64,2}, airline_code::String)

Función que hace el grafo de la red aérea dirigida.
"""
function my_betweenness_centrality(M::Array{Int64,2}, airline_code::String)
    
    grafo = DiGraph(M)
    res = betweenness_centrality(grafo)
    
    return res
    
end

In [ ]:
ggg= betweenness_centrality(jh)

In [ ]:
sort(ggg)

In [ ]:
"""
    plot_betweenness(M::Array{Int64,2}, airline_code::String)

Función que hace el grafo de la red aérea dirigida.
"""
function plot_betweenness(M::Array{Int64,2}, airline_code::String)
    
    aerolinea_matriz = string(airline_code, "_matriz_du.csv")
    nombres_multiplex = airport_names("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/$aerolinea_matriz")
    
    grafo = DiGraph(M)
    between = betweenness_centrality(grafo)
    orden = sort(between)
    
    xs = [nombres_multiplex]
    
    return red
    
end

In [ ]:
betweenness_centrality(g)

## Datos


### Aerolínea 2B

In [ ]:
lista_aerolineas = readdlm("/Users/yuriko/Documents/UNAM/Tesis_Lic/data/aerolineas_matrices/lista_aerolineas.csv", ',', String)
airline_code = lista_aerolineas[2]

In [ ]:
typeof(airline_code)

In [ ]:
aerolinea_matriz = string(airline_code, "_matriz_du.csv")
multiplex = import_matrix("data/aerolineas_matrices/$aerolinea_matriz")

In [ ]:
data_names_multiplex = airport_names("data/aerolineas_matrices/$aerolinea_matriz")

## Distribuciones de grado

### Multiplex Global

#### Grado de entrada

In [ ]:
in_degree_multiplex = in_degree(multiplex)

In [ ]:
distrib_in_multiplex = distrib_in_degree(multiplex)

In [ ]:
in_M = plot_distrib(multiplex, "in", "Multiplex Distribución Entrada", "black")

#### Grado de salida

In [ ]:
out_degree_multiplex = out_degree(multiplex)

In [ ]:
distrib_out_multiplex = distrib_out_degree(multiplex)

In [ ]:
out_M = plot_distrib(multiplex, "out", "Multiplex Distribución Salida", "black")

## Vecinos

### Multiplex global

In [ ]:
vecinos_M = @time neighbours(multiplex)

## Caminatas aleatorias

### Multiplex

In [ ]:
caminata_M = @time random_walk(multiplex, 25, 5) #nuevo

## Longitud de todas las iteraciones de caminatas 

In [ ]:
li_M = @time walk_length(caminata_M)

## Longitud promedio de las iteraciones de caminatas de cada nodo inicial

In [ ]:
lpn_M = @time walk_av_length(caminata_M)

In [ ]:
plot_av_length_vertex(caminata_M, "2B")

## Longitud promedio de caminatas por capa

In [ ]:
wl_M = @time walk_layer_length(caminata_M)

## Nodos visitados específicamente

In [ ]:
#No lo corro ahora para que no se tarde tanto
nv_M = @time visited_vertices(caminata_M)

In [ ]:
@time walk_names(caminata_M, "2B", 2)

In [ ]:
for i in 1:length(caminata_M)
    cam = @time walk_names(caminata_M, "2B", i)
    return cam
end

## Frecuencia de visitas

In [ ]:
freq_visit_M = @time freq_vertices_visits(caminata_M)

In [ ]:
distrib_visit_M = @time distrib_vertices_visits(caminata_M)

In [ ]:
plot_visit_freq(caminata_M);

In [ ]:
freq_vertex_visits(caminata_M, 1)

In [ ]:
distrib_vertex_visits(caminata_M, 1)

In [ ]:
plot_visit_freq_vertex(caminata_M, 1)

In [ ]:
distrib_prom_M = @time distrib_av_visits(caminata_M)

In [ ]:
plot_freq_av_visits(caminata_M, "2B")

## Cobertura (nodos distintos)

In [ ]:
cobi_M = @time coverage(caminata_M)

## Cobertura promedio de las caminatas de cada aeropuerto inicial

In [ ]:
cobs_M = @time vertex_coverage(caminata_M)

In [ ]:
plot_vertex_coverage(caminata_M, "2B")

## Cobertura promedio

In [ ]:
cobp_M = @time layer_coverage(caminata_M)

## Longitudes máximas de caminatas

In [ ]:
lmax_M = @time max_length(caminata_M)

In [ ]:
lMax_M = @time Max_length(caminata_M)

In [ ]:
lavmax_M = @time av_max_length(caminata_M)

## Longitudes mínimas de caminatas

In [ ]:
lmin_M = @time min_length(caminata_M)

In [ ]:
lMin_M = @time Min_length(caminata_M)

In [ ]:
lavmin_M = @time av_min_length(caminata_M)

## Endpoints

In [ ]:
endpts_M = @time endpoints(caminata_M)

In [ ]:
av_endpts_M = @time av_vertex_endpoints(caminata_M)

In [ ]:
distrib_endpts_M = @time distrib_endpoints(caminata_M)

In [ ]:
which_endpoints_M = @time which_endpoints(multiplex)

In [ ]:
freq_all_endpoints_M = @time freq_all_endpoints(caminata_M)

In [ ]:
#Falta función plot con nombres